# Test model

In [1]:
from keras.models import model_from_json
from pickle import dump, load
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from nltk.translate.bleu_score import sentence_bleu

Using TensorFlow backend.


In [2]:
images = '../../datasets/flickr8k/Flicker8k_Dataset/'
max_length=34

In [3]:
def load_descriptions(filename):
    file = open(filename, 'r')
    doc = file.read()
    file.close()
    
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    return mapping

In [4]:
def load_model(filename, verbose=False):
    json_file = open(filename + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(filename + ".h5")
    if verbose:
        print("The following model " + filename + " has been loaded")
    return loaded_model

In [39]:
model = load_model('./model_weights/model_7')

In [6]:
with open("./encoded_test_images.pkl", "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [7]:
with open("./wordtoix.pkl", "rb") as encoded_pickle:
    wordtoix = load(encoded_pickle)

In [8]:
with open("./ixtoword.pkl", "rb") as encoded_pickle:
    ixtoword = load(encoded_pickle)

In [9]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
#         print(sequence)
        sequence = pad_sequences([sequence], maxlen=max_length)
#         print(sequence)
        yhat = model.predict([photo,sequence], verbose=0)
#         print(yhat)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
len(encoding_test)

In [ ]:
#z+=1
z=18
pic = list(encoding_test.keys())[z]
image_feature_vector = encoding_test[pic].reshape((1,2048))
x=plt.imread(images+pic)
plt.imshow(x)
plt.show()
print("Greedy:",greedySearch(image_feature_vector))

In [ ]:
#z+=1
z=16
list(encoding_test.keys())[z]

In [ ]:
#z+=1
z=0
pic = list(encoding_test.keys())[z]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(images+pic)
plt.imshow(x)
plt.show()
print("Greedy:",greedySearch(image))

In [ ]:
image

In [ ]:
z=0
image_name = list(encoding_test.keys())[z]
image_feature_vector = encoding_test[image_name].reshape((1,2048))
candidate = greedySearch(image_feature_vector)

reference =[]
#     list_of_lists(test_descriptions[pic[:-4]])
for l in list_of_lists(test_descriptions[image_name[:-4]]):
    reference.append(l[0].split()[1:-1])
candidate = candidate.split()
score = sentence_bleu(reference, candidate)

In [ ]:
ixtoword[5180]

In [ ]:
test_descriptions['2869491449_1041485a6b']

In [ ]:
len(encoding_test)

In [40]:
test_descriptions = load_descriptions('test_captions.txt')
total_score = 0
for index in range(len(encoding_test)):
    image_name = list(encoding_test.keys())[index]
    image_feature_vector = encoding_test[image_name].reshape((1,2048))
    candidate = greedySearch(image_feature_vector)

    reference =[]
#     list_of_lists(test_descriptions[pic[:-4]])
    for l in list_of_lists(test_descriptions[image_name[:-4]]):
        reference.append(l[0].split()[1:-1])
    candidate = candidate.split()
    score = sentence_bleu(reference, candidate,weights=(1, 0, 0, 0))
    total_score+=score
    print(total_score)
print(total_score/1000)

0.2857142857142857
1.0357142857142856
1.5310730855715322
2.0644064189048654
2.17471853172794
2.7302740872834956
3.1052740872834956
3.688607420616829
3.7997185317279403
4.244162976172385
5.077496309505718
5.577496309505718
5.910829642839051
6.110829642839051
6.99971853172794
7.593305860377476
8.03775030482192
8.63775030482192
9.20917887625049
9.780607447679062
10.113940781012396
10.477577144648759
11.074136689191672
11.199136689191672
11.532470022525006
11.932470022525006
12.056309722489319
12.364002030181627
12.51784818402778
13.351181517361114
13.934514850694448
14.434514850694448
15.059514850694448
15.63094342212302
16.255943422123018
16.55594342212302
16.78671265289225
17.186712652892247
17.986712652892248
18.31420495412344
18.81420495412344
19.385633525552013
19.68391329782347
19.98391329782347
20.23391329782347
20.98391329782347
21.599297913208087
21.723137613172398
22.152693412416756
22.5971378568612
23.041582301305642
23.874915634638974
24.27262199766758
24.93928866433425
25.272

199.87473428395188
200.44616285538046
200.74616285538048
201.24616285538048
201.68741130667277
202.24296686222831
202.46518908445054
202.96518908445054
203.426727545989
203.87117199043345
204.2045053237668
204.90450532376678
205.6187910380525
206.04736246662392
206.49180691106838
207.23484511085425
207.95871504528301
208.26640735297534
208.66411371600395
209.16411371600395
209.7196692715595
210.10302111098838
210.18635444432172
210.90064015860744
211.2339734919408
211.5372388217971
211.5372388217971
211.5372388217971
212.10155997172419
212.8158456860099
213.3526151810637
213.71088083635058
214.15532528079504
214.75532528079503
215.31088083635058
215.59659512206485
215.91865681909712
216.41865681909712
216.71865681909713
217.21865681909713
217.81865681909713
218.24189768154244
218.63129807307814
219.12253652492493
219.78920319159158
220.07491747730586
220.25588496091305
220.3987421037702
220.81540877043685
220.81540877043685
221.41540877043684
222.11139490573692
222.66695046129246
223.5

390.3814945818908
390.49305966196505
390.9597263286317
391.4597263286317
392.1263929952984
392.52639299529835
392.82467276756984
393.03938056559133
393.706047232258
394.53938056559133
395.4062584653415
395.96181402089707
396.5173695764526
397.07292513200815
397.07292513200815
397.41656977140366
397.6372974361065
398.0372974361065
398.5326562359637
398.865989569297
399.5932622965698
399.7879624923376
400.00858671798375
400.5641422735393
401.3641422735393
402.078427987825
402.41176132115834
403.1117613211583
403.3617613211583
403.56061450267265
404.01303321169064
404.54685814696853
405.0406391531547
405.1517502642658
405.29460740712295
406.0376456069088
406.7299532992165
407.11456868383186
407.2339905689275
407.789546124483
408.3538672744101
408.59196251250535
409.1475180680609
409.7475180680609
409.9141847347276
410.1641847347276
410.7641847347276
411.62132759187045
412.12132759187045
412.62132759187045
413.45466092520377
413.70466092520377
414.20466092520377
414.70466092520377
415.5935

In [19]:
z=0
image_name = list(encoding_test.keys())[z]
image_feature_vector = encoding_test[image_name].reshape((1,2048))
candidate = greedySearch(image_feature_vector)

reference =[]
#     list_of_lists(test_descriptions[pic[:-4]])
for l in list_of_lists(test_descriptions[image_name[:-4]]):
    reference.append(l[0].split()[1:-1])
candidate = candidate.split()
score = sentence_bleu(reference, candidate)

In [29]:
[[]].append(reference)

In [26]:
reference

In [20]:
candidate

['black', 'dog', 'is', 'running', 'through', 'the', 'grass']

In [17]:
score

6.968148412761692e-155

In [11]:
def list_of_lists(lst): 
    res = [] 
    for el in lst: 
        sub = el.split(', ') 
        res.append(sub) 
      
    return(res) 


In [ ]:
score

In [ ]:
image_name

In [ ]:
candidate

In [ ]:
reference